In [1]:
import pandas as pd
import numpy as np

from cryptodatapy.extract.datarequest import DataRequest
from cryptodatapy.extract.getdata import GetData
from cryptodatapy.transform.od import OutlierDetection
from cryptodatapy.transform.impute import Impute
from cryptodatapy.transform.filter import Filter
from cryptodatapy.transform.clean import CleanData, stitch_dataframes
from cryptodatapy.transform.impute import Impute

fatal: bad revision 'HEAD'
Importing plotly failed. Interactive plots will not work.


In [2]:
# get all Binance perp futures tickers
data_req = DataRequest(source='ccxt')
perp_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binanceusdm', as_list=True)

In [3]:
len(perp_tickers)

314

In [4]:
# get Binance spot tickers
data_req = DataRequest(source='ccxt')
spot_tickers = GetData(data_req).get_meta(method='get_markets_info', exch='binance', as_list=True)

In [5]:
# find intersecting tickers
binance_tickers = [ticker for ticker in perp_tickers if ticker in spot_tickers]

In [6]:
# number of tickers
len(binance_tickers)

314

In [7]:
## # get cryptocompare tickers
data_req = DataRequest(source='cryptocompare')
cc_tickers = GetData(data_req).get_meta(method='get_assets_info', as_list=True)

In [8]:
# keep only USDT ticker
bin_tickers = []
for ticker in binance_tickers:
    if '/' in ticker and ticker.split('/')[1] == 'USDT':
        bin_tickers.append(ticker.split('/')[0])

In [9]:
# usdt tickers
usdt_tickers = [ticker.split('/')[0] for ticker in binance_tickers if '/'in ticker and ticker.split('/')[1] == 'USDT']

In [10]:
# intersecting tickers
tickers = [ticker for ticker in usdt_tickers if ticker in cc_tickers]

In [11]:
len(tickers)

0

### Binance Perp Futures

In [12]:
# pull daily OHLC and funding rates for perp futures on Binance USDM exchange
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume', 'funding_rate'], 
                       mkt_type='perpetual_future', 
                       freq='d')

In [13]:
# df1 = GetData(data_req).get_series()

In [14]:
# df1.to_csv('binance_perp_futures.csv')
df1 = pd.read_csv('../../../../factorlab/notebooks/binance_perp_futures.csv', index_col=['date', 'ticker'], parse_dates=['date'])

In [15]:
df1.head()

,,open,high,low,close,volume,funding_rate
date,ticker,,,,,,
2019-09-08,BTC,10000.00,10412.65,10000.00,10391.63,3096.291,NaN
2019-09-09,BTC,10316.62,10475.54,10077.22,10307.00,14824.373,NaN
2019-09-10,BTC,10307.00,10382.97,9940.87,10102.02,9068.955,0.0002
2019-09-11,BTC,10094.27,10293.11,9884.31,10159.55,10897.922,0.0003
2019-09-12,BTC,10163.06,10450.13,10042.12,10415.13,15609.634,0.0003


### Binance Spot

In [16]:
# pull OHLC from Binance
data_req = DataRequest(source='ccxt',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [17]:
# df2 = GetData(data_req).get_series()

In [18]:
# df2.to_csv('binance_spot.csv')
df2 = pd.read_csv('../../../../factorlab/notebooks/binance_spot.csv', index_col=['date', 'ticker'], parse_dates=['date'])

In [19]:
df2.head()

open     high      low    close       volume
date       ticker                                                 
2017-08-17 BTC     4261.48  4485.39  4200.74  4285.08   795.150377
           ETH      301.13   312.18   298.00   302.00  7030.710340
2017-08-18 BTC     4285.08  4371.52  3938.77  4108.37  1199.888264
           ETH      302.00   311.79   283.94   293.96  9537.846460
2017-08-19 BTC     4108.37  4184.69  3850.00  4139.98   381.309763

### CryptoCompare - Historical Prices

In [20]:
# pull OHLC from CryptoCompare
data_req = DataRequest(source='cryptocompare',
                       tickers=tickers, 
                       fields=['open', 'high', 'low', 'close', 'volume'], 
                       freq='d')

In [21]:
# df3 = GetData(data_req).get_series()

In [22]:
# df3.to_csv('cc_spot.csv')
df3 = pd.read_csv('../../../../factorlab/notebooks/cc_spot.csv', index_col=['date', 'ticker'], parse_dates=['date'])

### Clean Data

In [23]:
df = stitch_dataframes([df1, df2, df3])
df.funding_rate = df.funding_rate.fillna(0)

In [24]:
df.head()

,,open,high,low,close,volume,funding_rate
date,ticker,,,,,,
2010-07-17,BTC,0.04951,0.04951,0.04951,0.04951,20.00,0.0
2010-07-18,BTC,0.04951,0.08585,0.04951,0.08584,75.01,0.0
2010-07-19,BTC,0.08584,0.09307,0.07723,0.08080,574.00,0.0
2010-07-20,BTC,0.08080,0.08181,0.07426,0.07474,262.00,0.0
2010-07-21,BTC,0.07474,0.07921,0.06634,0.07921,575.00,0.0


In [25]:
delisted_tickers = ['AGIX', 'CTK', 'CVC', 'CVX', 'DGB', 'FTT', 'GLMR', 'IDEX', 'MDT',
       'OCEAN', 'RAD', 'RAY', 'SC', 'SLP', 'SNT', 'STPT', 'STRAX', 'WAVES']

In [26]:
# clean data
clean_df = CleanData(df).filter_delisted_tickers().\
                         filter_min_nobs(ts_obs=1500, cs_obs=10).\
                         filter_outliers(od_method='mad', excl_cols=['volume', 'funding_rate'], thresh_val=10).\
                         repair_outliers(imp_method='fcst').\
                         filter_avg_trading_val(thresh_val=1000000).\
                         filter_missing_vals_gaps().\
                         get(attr='df').dropna(how='all')

Index(['AGIX', 'CTK', 'CVC', 'CVX', 'DGB', 'FTT', 'GLMR', 'IDEX', 'MDT',
       'OCEAN', 'RAD', 'RAY', 'SC', 'SLP', 'SNT', 'STPT', 'STRAX', 'WAVES'],
      dtype='object', name='ticker')


In [27]:
clean_df.loc[:, 'OCEAN', :]

KeyError: 'OCEAN'

In [28]:
clean_df.loc[:, 'BTC', :]

,open,high,low,close,volume,funding_rate
date,,,,,,
2015-06-12,229.88,231.58,229.29,230.46,40744.820,0.000000
2015-06-13,230.46,233.14,229.01,232.48,38008.730,0.000000
2015-06-14,232.48,235.51,232.09,233.75,32894.870,0.000000
2015-06-15,233.75,238.55,233.29,237.0,63467.090,0.000000
2015-06-16,237.0,254.15,235.7,249.82,122473.610,0.000000
...,...,...,...,...,...,...
2024-07-31,66159.3,66826.3,64500.4,64601.8,246389.446,0.000141
2024-08-01,64601.8,65650.0,62271.2,65328.9,372654.590,0.000282
2024-08-02,65329.0,65577.0,61200.2,61483.7,421628.420,0.000300


In [29]:
clean_df.to_parquet('s3://factorlab-data/binance_historical_ohlcv_daily.parquet')

In [30]:
clean_df.to_parquet('../../../../factorlab/notebooks/binance_historical_ohlcv_daily.parquet')

In [31]:
clean_df.to_csv('../../../../factorlab/notebooks/binance_historical_ohlcv_daily.csv')

In [32]:
clean_df.to_csv('s3://factorlab-data/binance_historical_ohlcv_daily.csv')